# Plot the posterior mass-radius countours

Here is an example of how to obtain the posteriro mass-radius countours from the posterior distribution of the EOS parameters.

After obtaining the distributions of the EOS parameters, we can map the posterior distributions of the specific EOS in mass-radius (M-R) space. This procedure helps us to understand how observational constraints affect the EOS. Each point in the EOS parameter space is uniquely correlated with a point in the EOS posterior parameter space. Then, by varying the central density, EOS points can be mapped onto the M-R plane by deriving the Tolman-Oppenheimer-Volkoff (TOV) equations.

Here is a step-by-step explanation.

First import all the package that will be used.

In [ ]:
import numpy as np 
from scipy.constants import pi
from scipy.integrate import ode
from scipy.interpolate import interp1d
import random
import matplotlib.pyplot as plt
import seaborn as sns 

The constans that we will used in the following:

In [2]:
c = 3e10  # Speed of light in cm/s
G = 6.67428e-8    # Gravitational constant in cm^3/g/s^2 or dyne cm^2/g^2
Msun = 1.989e33   # Solar mass in grams
dyncm2_to_MeVfm3 = 1. / (1.6022e33)  # Conversion factor from dyn/cm^2 to MeV/fm^3
gcm3_to_MeVfm3 = 1. / (1.7827e12)    # Conversion factor from g/cm^3 to MeV/fm^3
oneoverfm_MeV = 197.33    # Conversion factor for fm to MeV

To obtain the corresponding maximum mass  index for each posterior EOS parameter.

In [ ]:
def EofP(P, epsgrid, presgrid):
   idx = np.searchsorted(presgrid, P)
   if idx == 0:
       eds = epsgrid[0] * pow(P / presgrid[0], 3. / 5.)
   if idx == len(presgrid):
       eds = epsgrid[-1] * pow(P / presgrid[-1], 3. / 5.)
   else:
       ci = np.log(presgrid[idx] / presgrid[idx-1]) /np.log(epsgrid[idx] /epsgrid[idx-1])
       eds = epsgrid[idx-1] * pow(P / presgrid[idx-1], 1. / ci)
   return eds


def EofP(P, epsgrid, presgrid):
   idx = np.searchsorted(presgrid, P)
   if idx == 0:
       eds = epsgrid[0] * pow(P / presgrid[0], 3. / 5.)
   if idx == len(presgrid):
       eds = epsgrid[-1] * pow(P / presgrid[-1], 3. / 5.)
   else:
       ci = np.log(presgrid[idx] / presgrid[idx-1]) /np.log(epsgrid[idx] /epsgrid[idx-1])
       eds = epsgrid[idx-1] * pow(P / presgrid[idx-1], 1. / ci)
   return eds
   
   
def TOV(r, y, inveos):
    pres, m = y
    eps = inveos(pres)
    if np.abs(4.*pi*r**3. * pres) >1e10 :
        return ([1e27,1e27])
    else:
        dpdr = -(eps + pres) * (m + 4.*pi*r**3. * pres)
        dpdr = dpdr/(r*(r - 2.*m))
        dmdr = 4.*pi*r**2.0 * eps
    return np.array([dpdr, dmdr])

def solveTOV(center_rho, energy_density, pressure):
    """Solve TOV equation from given Equation of state in the neutron star 
    core density range

    Args:
        center_rho(array): This is the energy density here is fixed in main
        that is np.logspace(14.3, 15.6, 50)
        energy_density (array): Desity array of the neutron star EoS, in MeV/fm^{-3}
        Notice here for simiplicity, we omitted G/c**4 magnitude, so 
        (value in MeV/fm^{-3})*G/c**4, could convert to the energy density we are
        using, please check the Test_EOS.csv to double check the order of magnitude.
        
        pressure (array): Pressure array of neutron star EoS, also in nautral unit
        with MeV/fm^{-3}, still please check the Test_EOS.csv, the conversion is 
        (value in dyn/cm3)*G/c**4.

    Returns:
        Mass (array): The array that contains all the Stars' masses, in M_sun as a 
        Units.
        Radius (array): The array that contains all the Stars's radius, in km.
    """
    c = 3e10
    G = 6.67428e-8
    Msun = 1.989e33

    unique_pressure_indices = np.unique(pressure, return_index=True)[1]
    unique_pressure = pressure[np.sort(unique_pressure_indices)]

    #Interpolate pressure vs. energy density
    eos = interp1d(energy_density, pressure, kind='cubic', fill_value='extrapolate')

    #Interpolate energy density vs. pressure
    inveos = interp1d(unique_pressure, energy_density[unique_pressure_indices], kind='cubic', fill_value='extrapolate')
    
    Pmin = pressure[20]
    r = 4.441e-16
    dr = 10.
    center_rho = center_rho * G/c**2.
    
    pcent = eos(center_rho)
    P0 = pcent - (2.*pi/3.)*(pcent + center_rho) *(3.*pcent + center_rho)*r**2.
    m0 = 4./3. *pi *center_rho*r**3.
    stateTOV = np.array([P0, m0])
    
    sy = ode(TOV, None).set_integrator('dopri5')
    
    #have been modified from Irida to this integrator
    sy.set_initial_value(stateTOV , r).set_f_params(inveos)
    
    while sy.successful() and stateTOV[0]>Pmin:
        stateTOV = sy.integrate(sy.t+dr)
        dpdr, dmdr = TOV(sy.t+dr, stateTOV, inveos)
        dr = 0.46 * (1./stateTOV[1] * dmdr - 1./stateTOV[0]*dpdr)**(-1.)

    return stateTOV[1]*c**2./G/Msun, sy.t/1e5  

def TOV1(r, y, grids):
   pres, m = y
   epsgrid = grids[0]
   presgrid = grids[1]
   eps = EofP(pres, epsgrid, presgrid)
   dpdr = -(eps + pres) * (m + 4.*pi*r**3. * pres)
   dpdr = dpdr/(r*(r - 2.*m))
   dmdr = 4.*pi*r**2.0 * eps
   return np.array([dpdr, dmdr])


def solveTOV1(rhocent, eps, pres):
   eps = np.array(eps)
   pres = np.array(pres)
   Pmin = pres[20]
   r = 4.441e-16
   dr = 10.
   rhocent = rhocent * G/c**2.
   pcent = PofE(rhocent, eps, pres)
   P0 = pcent - (2.*pi/3.)*(pcent + rhocent) *(3.*pcent + rhocent)*r**2.
   m0 = 4./3. *pi *rhocent*r**3.
   stateTOV = np.array([P0, m0])
   sy = ode(TOV1, None).set_integrator('dopri5')
   par = np.vstack([eps, pres])
   sy.set_initial_value(stateTOV, r).set_f_params(par)
   while sy.successful() and stateTOV[0]>Pmin:
       stateTOV = sy.integrate(sy.t+dr)
       dpdr, dmdr = TOV1(sy.t+dr, stateTOV, par)
       dr = 0.46 * (1./stateTOV[1] * dmdr - 1./stateTOV[0]*dpdr)**(-1.)

   return stateTOV[1]*c**2./G/Msun, sy.t/1e5


def compute_tov_properties(densities, eps_total, pres_total):
   max_length = len(densities)
   RFSU2R = np.zeros(max_length)
   MFSU2R = np.zeros(max_length)
   for i in range(max_length):
       MFSU2R[i], RFSU2R[i] = solveTOV1(densities[i], eps_total, pres_total)
       if i > 20 and MFSU2R[i] < MFSU2R[i - 1]:
           return MFSU2R[:i], RFSU2R[:i]  # Only return up to i, not the full arrays

   return MFSU2R, RFSU2R


Next, onw way is to sample the central energy density to obtain the posterior MR distribution. 

Another way is to sample the MR point directly from the MR relation. This is the same concept. 

The following code is an example of sampling the MR point directly using the posterior distribution of strangeon_matter_EOS.

In [ ]:
Ms = []
Rs = []

Nq=18
def Strangeon_compute_EOS(n, theta):
    epsilon, ns = theta
    A12 = 6.2
    A6 = 8.4 
    mq = 300  # Given constant value for mq
    
    sigma = np.sqrt(A6 / (2 * A12)) * (Nq / (3 * ns)) 
    energy_density = 2 * epsilon * (A12 * sigma**4 * n**5 - A6 * sigma**2 * n**3) + n * Nq * mq
    pressure = 4 * epsilon * (2 * A12 * sigma**4 * n**5 - A6 * sigma**2 * n**3)

    return energy_density * G / c**2 / gcm3_to_MeVfm3, pressure * G / c**4 / dyncm2_to_MeVfm3 


array_size = 50  # Replace array_size with the desired number of random numbers 

data = np.loadtxt('equal_weighted_post.txt', delimiter=' ',skiprows=1) 
Rs = np.zeros(len(data)*50)
Ms = np.zeros(len(data)*50)
massradius=[]
for i in range(0,len(data)):
   theta = data[i][:2]  # The first two columns are epsilon/Nq, ns
   n_min = 3 * theta[1] / Nq  
   n_max = 0.16 * 8 * 3 / Nq    
   nbar_values = np.linspace(n_min, n_max, 10000)  # 10000 points between n_min and n_max  
   energy_densities, pressures = Strangeon_compute_EOS(nbar_values, theta)
   density = np.logspace(14.3, 15.6, 50)
   max_length =  len(density)

   MFSU2R, RFSU2R = compute_tov_properties(density, energy_densities,pressures)
   mr = interp1d(MFSU2R, RFSU2R)
   index = len(MFSU2R) 
   for j in range(0,50):
        Mpoint = max(MFSU2R)*random.random()
#A random mass Mpoint is sampled uniformly between 0 and the maximum mass in MFSU2R.
        Rpoint = mr(Mpoint) 
        Ms[i+j] =  Mpoint
        Rs[i+j] = Rpoint
#The random mass and radius values are stored in the Ms and Rs arrays.
        massradius.append([Rs[i],Ms[i]])
        print(i,Mpoint,Rpoint) 
        
plt.scatter(Rs, Ms, color='palevioletred')   

np.savetxt('withNICERtwo_para_Mass.txt',Ms)  
np.savetxt('withNICERtwo_para_Radius.txt',Rs)
np.savetxt('withNICERtwo_para_MR.txt',massradius) 




Then, you can use the MR points to plot the MR contour at the 99.7% confidence level.

In [ ]:
X =np.array(np.loadtxt('withNICERtwo_para_Radius.txt') )
Y=np.array(np.loadtxt('withNICERtwo_para_Mass.txt')) 
R_l = []
M_l = []
MR = list(zip(X, Y))
MR = [m for m in MR if  0.001< m[0] <16]
R_l, M_l = zip(*MR)
   

fig, ax = plt.subplots(figsize=(8, 6))
sns.kdeplot(x=R_l, y=M_l, shade=False,levels=[0.03], linestyles='--',linewidths=1.,
             cmap=None, alpha=0.9, colors=['green'], ax=ax)   
plt.scatter(R_l, M_l, color='palevioletred')  

ax.set_xlim(6, 18)
ax.set_ylim(1, 5)
ax.set_xlabel("$R~(\mathrm{km})$")
ax.set_ylabel(r"$M \ (M_{\odot}) $")
ax.tick_params(direction='in', top='on', right='on', which='both')
ax.set_title('M-R Posterior Distribution with Contour Levels')
plt.savefig("MR_posterior.pdf")
plt.show()

